In [1]:
!nvidia-smi

Fri Apr 26 11:08:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.98                 Driver Version: 535.98       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A30                     Off | 00000000:3B:00.0 Off |                    0 |
| N/A   22C    P0              24W / 165W |     18MiB / 24576MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
### REDO classification report based on predictions saved

import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
print(os.environ["CUDA_VISIBLE_DEVICES"])

0


In [40]:
from pathlib import Path
import re

import numpy as np
import pandas as pd

from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import multilabel_confusion_matrix, classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [41]:
DATA_PATH = Path("/gaueko0/users/nmishra/multiling_fludetection/evals/eval_revisedcateg/testset0.6_0.2_0.2")
folder_name = 'one_to_many_lcpredictions_run5'
DATA_PATH = DATA_PATH.joinpath(folder_name)
DATA_PATH

PosixPath('/gaueko0/users/nmishra/multiling_fludetection/evals/eval_revisedcateg/testset0.6_0.2_0.2/one_to_many_lcpredictions_run5')

In [44]:
models = ['twitter-xlm-roberta-base','bernice', 'bert-base-multilingual', 'mdeberta', 'xlm-roberta-base']
labels_map = {0:'1. Likely ILI infection', 
              # 1:'2. Likely COVID-19 Infection (after 2020 only)', 
              1:'3. Not Related to ILI or COVID-19 Infection',
              # 3:'4. Ambiguous/Unsure'
             }
labels_map.keys()

dict_keys([0, 1])

In [45]:
for n in range(len(models)):
    file_names = [i for i in DATA_PATH.glob('*.csv') if i.stem.startswith(models[n])]
    # print(file_names)
    
    languages = [re.findall('_(.+?)_', i.stem)[0] for i in file_names]
    print(languages)
    
    dfs = [pd.read_csv(i) for i in file_names]
    print(len(dfs))
    
    lang_idx = {i:j['index'].values for i,j in zip(languages, dfs)}
    for lang, df in zip(languages, dfs):
        print(file_names[n])
        print(f"{models[n]} trained on {lang}")
        lang_df = df[df['index']==lang_idx[lang]]
        y = lang_df.iloc[:,2]
        y_pred = lang_df.iloc[:,1]
    
        print(round(f1_score(y, y_pred, average='weighted'),2))
        print(round(f1_score(y, y_pred, average='macro'),2))
        print(round(accuracy_score(y, y_pred),2))

['it', 'de', 'fr', 'es', 'en']
5
/gaueko0/users/nmishra/multiling_fludetection/evals/eval_revisedcateg/testset0.6_0.2_0.2/one_to_many_lcpredictions_run5/twitter-xlm-roberta-base_it_predictions.csv
twitter-xlm-roberta-base trained on it
0.76
0.74
0.77
/gaueko0/users/nmishra/multiling_fludetection/evals/eval_revisedcateg/testset0.6_0.2_0.2/one_to_many_lcpredictions_run5/twitter-xlm-roberta-base_it_predictions.csv
twitter-xlm-roberta-base trained on de
0.78
0.78
0.78
/gaueko0/users/nmishra/multiling_fludetection/evals/eval_revisedcateg/testset0.6_0.2_0.2/one_to_many_lcpredictions_run5/twitter-xlm-roberta-base_it_predictions.csv
twitter-xlm-roberta-base trained on fr
0.79
0.78
0.79
/gaueko0/users/nmishra/multiling_fludetection/evals/eval_revisedcateg/testset0.6_0.2_0.2/one_to_many_lcpredictions_run5/twitter-xlm-roberta-base_it_predictions.csv
twitter-xlm-roberta-base trained on es
0.78
0.77
0.78
/gaueko0/users/nmishra/multiling_fludetection/evals/eval_revisedcateg/testset0.6_0.2_0.2/one_to

In [46]:
n = 4
file_names = [i for i in DATA_PATH.glob('*.csv') if i.stem.startswith(models[n])]
print([i.stem for i in file_names])

languages = [re.findall('_(.+?)_', i.stem)[0] for i in file_names]
print(languages)

dfs = [pd.read_csv(i) for i in file_names]
print(len(dfs))

lang_idx = {i:j['index'].values for i,j in zip(languages, dfs)}

['xlm-roberta-base_it_predictions', 'xlm-roberta-base_fr_predictions', 'xlm-roberta-base_en_predictions', 'xlm-roberta-base_de_predictions', 'xlm-roberta-base_es_predictions']
['it', 'fr', 'en', 'de', 'es']
5


In [47]:
lang = 'de'
df = dfs[languages.index(lang)]

print(f"{models[n]} trained on {lang}")
lang_df = df[df['index']==lang_idx[lang]]
y = lang_df.iloc[:,2]
y_pred = lang_df.iloc[:,1]

print(round(f1_score(y, y_pred, average='weighted'),2))
print(round(f1_score(y, y_pred, average='macro'),2))
print(round(accuracy_score(y, y_pred),2))

xlm-roberta-base trained on de
0.79
0.79
0.79


In [48]:
cs = classification_report(y, y_pred, 
                           labels=list(labels_map.keys()), 
                           target_names=labels_map.values(), 
                           output_dict=True
                          )
cs = pd.DataFrame(cs).transpose().round(2)
cs

,precision,recall,f1-score,support
1. Likely ILI infection,0.70,0.84,0.76,341.00
3. Not Related to ILI or COVID-19 Infection,0.88,0.76,0.82,519.00
accuracy,0.79,0.79,0.79,0.79
macro avg,0.79,0.80,0.79,860.00
weighted avg,0.81,0.79,0.79,860.00


In [49]:
cm

array([[285,  56],
       [123, 396]])

In [50]:
pd.DataFrame(cm).to_dict()

{0: {0: 285, 1: 123}, 1: {0: 56, 1: 396}}

In [52]:
# if lang =='en':
#     lbls_map = {k:v for k,v in labels_map.items() if '2. Likely COVID-19' not in v}
#     print(lbls_map)
# else:
#     lbls_map = labels_map

lbls_map = {0:'1. Likely ILI infection',
            2:'3. Not Related to ILI or COVID-19 Infection'}
cm = confusion_matrix(y, y_pred, normalize='true').round(2)
cmtx = pd.DataFrame(cm,
                    index=list(lbls_map.values()),
                    columns=list(lbls_map.values())
                   )
cmtx

,1. Likely ILI infection,3. Not Related to ILI or COVID-19 Infection
1. Likely ILI infection,0.84,0.16
3. Not Related to ILI or COVID-19 Infection,0.24,0.76


Model trained on all tested on each

In [13]:
file_names = [i for i in DATA_PATH.glob('*.csv') if 'all' in i.stem]
for file in file_names:
    print(f"{file}")
    for lang, idx in lang_idx.items():
        if lang != 'all':
            print(f"{lang}")
            df = pd.read_csv(file)
            lang_df = df[df['index'].isin(lang_idx[lang])]
            y = lang_df.iloc[:,2]
            y_pred = lang_df.iloc[:,1]
            if lang =='en':
                print(round(f1_score(y, y_pred, labels=[0,2,3], average='weighted'),2))
                print(round(f1_score(y, y_pred, labels=[0,2,3], average='macro'),2))
            else: 
                print(round(f1_score(y, y_pred, average='weighted'),2))
                print(round(f1_score(y, y_pred, average='macro'),2))
            print(round(accuracy_score(y, y_pred),2))

In [14]:
file_names = [i for i in DATA_PATH.glob('*.csv') if 'all' in i.stem]
n = 4
file_names[n]

IndexError: list index out of range

In [15]:
lang = 'it'
df = pd.read_csv(file_names[n])
lang_df = df[df['index'].isin(lang_idx[lang])]
y = lang_df.iloc[:,2]
y_pred = lang_df.iloc[:,1]
print(lang)
cs = classification_report(y, y_pred, 
                           labels=list(labels_map.keys()), 
                           target_names=labels_map.values(), 
                           output_dict=True
                          )
cs = pd.DataFrame(cs).transpose().round(2)
cs

IndexError: list index out of range

In [19]:
if lang =='en':
    lbls_map = {k:v for k,v in labels_map.items() if '2. Likely COVID-19' not in v}
    print(lbls_map)
else:
    lbls_map = labels_map
cm = confusion_matrix(y, y_pred, normalize='true').round(2)
cmtx = pd.DataFrame(cm,
                    index=list(labels_map.values()),
                    columns=list(labels_map.values())
                   )
cmtx

,1. Likely ILI infection,2. Likely COVID-19 Infection (after 2020 only),3. Not Related to ILI or COVID-19 Infection,4. Ambiguous/Unsure
1. Likely ILI infection,0.75,0.01,0.18,0.06
2. Likely COVID-19 Infection (after 2020 only),0.20,0.20,0.60,0.00
3. Not Related to ILI or COVID-19 Infection,0.17,0.00,0.76,0.06
4. Ambiguous/Unsure,0.46,0.08,0.15,0.31
